In [1]:
# Dependencies
from pprint import pprint
import pymongo
import tweepy
import json
import numpy as np
from twtkey import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

#connection with Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.project_reax
db_cities_full = db.cities_full

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
list_city_ids = []
results = db_cities_full.find()
for result in results:
    try:
        list_city_ids.append(result["woeids"][0]["Id"])
    except:
        pass

In [ ]:
def Removedups(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

list_city_ids = Removedups(list_city_ids)

In [ ]:
# Top Twitter Trends for each City

twitter_trends = []

for cityid in list_city_ids:
    try:
        trends1 = api.trends_place(cityid)
        twitter_trends.append(trends1)
    except:
        pass

In [ ]:
for trend in twitter_trends:
    for trendelement in trend:
        db.twitter_trends.insert_one(trendelement)

In [ ]:
# Getting top trends in US (country overall)
#trends1 = api.trends_place(23424977)
db.twitter_trends.insert_one(trends1[0])
#pprint(trends1[0])

In [ ]:
pipeline = [
     {"$lookup": {"from": "city_full_lite", "localField": "locations.woeid", "foreignField": "Id", "as": "coordinates"}}]
     
db.twitter_trends_full.insert_many(db.twitter_trends.aggregate(pipeline))

In [5]:
trend_woeid = 23424977

In [51]:
i = 0

for trend in db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"]:
    i = i+1
    if (i==6):
        break
    if (i<4):
        continue
        
    trend_name = trend["name"]

    for location in db.twitter_trends_full.find():
    
        try:
            lat = location["coordinates"][0]["latitude"]
            lng = location["coordinates"][0]["longitude"]
            city = location["coordinates"][0]["city"]
            state = location["coordinates"][0]["state"]
            woeid = location["coordinates"][0]["Id"]
    
            geocode = str(lat) + "," + str(lng) + ",50mi"
            public_tweets = api.search(trend_name, tweet_mode="extended", count=10, result_type="recent", geocode=geocode)
    
            db.twitter_trends_full.update_one(
            {"locations.woeid": trend_woeid, 'trends.name': trend_name},
            { "$push": 
               {"trends.$.public_tweets": {
                   "woeid": woeid,
                   "latitude": lat,
                   "longitude": lng,
                   "state": state,
                   "city": city,
                   "tweets": public_tweets
               }}
            }
            )
        except:
            pass


In [106]:
# Run Vader Analysis on each tweet

i = 0

for trend in db.twitter_trends_full.find({"locations.woeid": trend_woeid})[0]["trends"]:
    i = i+1
    if (i==2):
        break
        
    trend_name = trend["name"]
        
    for tweet_location in trend["public_tweets"]:
        woeid = tweet_location["woeid"]
        vader_scores=[]
        for tweet in tweet_location['tweets']['statuses']:
            try:
                score = analyzer.polarity_scores(tweet["retweeted_status"]["full_text"])
            except:
                score = analyzer.polarity_scores(tweet["full_text"])
            #if score["neu"] != 1:
            vader_scores.append(score["compound"]) 
        #print (str(trend_woeid) + "," + str(trend_name) +"," +str(woeid))
        
        
        db.twitter_trends_full.update_one(
            {"locations.woeid": trend_woeid, "trends.name": trend_name, "trends.public_tweets.woeid": woeid},
            { "$set":
                #{"trends": {"public_tweets" : {"tweets": {"vader_scores": vader_scores}}}}
                {"trends.$.public_tweets.$[elem].vader_scores": vader_scores}
            }
            ,array_filters=[{"elem.woeid": woeid }]
            )